In [4]:
from pydantic import BaseModel, EmailStr, validator
from typing import List, Optional, Callable
import functools

In [6]:
!pip install email-validator

In [7]:
from pydantic import BaseModel, EmailStr, field_validator
from typing import List, Optional

class Book(BaseModel):
    title: str
    author: str
    year: int
    available: bool
    categories: Optional[List[str]] = []

    @field_validator('categories', mode="before")
    def category_must_be_str(cls, v):
        if v is None:
            return []
        if not all(isinstance(item, str) for item in v):
            raise ValueError('Category must be a string')
        return v

class User(BaseModel):
    name: str
    email: EmailStr
    membership_id: str


In [8]:
class Library(BaseModel):
    books: List[Book]
    users: List[User]

    def total_books(self) -> int:
        return len(self.books)


In [9]:
def add_book(library: Library, book: Book) -> None:
    library.books.append(book)

def find_book(library: Library, title: str) -> Optional[Book]:
    for book in library.books:
        if book.title == title:
            return book
    return None

def is_book_borrow(library: Library, title: str) -> bool:
    book = find_book(library, title)
    if book is None or not book.available:
        raise BookNotAvailable(f"Book '{title}' is not available!")
    return not book.available

def return_book(library: Library, title: str) -> bool:
    book = find_book(library, title)
    if book and not book.available:
        book.available = True
        return True
    return False


In [10]:
class BookNotAvailable(Exception):
    pass

def log_operation(func: Callable) -> Callable:
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print(f"Operation {func.__name__} called with args={args}, kwargs={kwargs}")
        return func(*args, **kwargs)
    return wrapper


In [11]:
lib = Library(books=[], users=[])
book = Book(title="Python", author="Guido", year=2020, available=True, categories=["Programming"])
user = User(name="Alice", email="alice@example.com", membership_id="123")
add_book(lib, book)
lib.users.append(user)
print(lib.total_books())


1


In [12]:
add_book(lib, Book(title="AI", author="John", year=2022, available=True, categories=["Science"]))
print(lib.total_books())      # должно стать 2


2
